In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("diamonds.csv")

In [6]:
df.shape

(53940, 11)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   carat       53940 non-null  float64
 2   cut         53940 non-null  object 
 3   color       53940 non-null  object 
 4   clarity     53940 non-null  object 
 5   depth       53940 non-null  float64
 6   table       53940 non-null  float64
 7   price       53940 non-null  int64  
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB


In [7]:
df = df.drop('Unnamed: 0',axis=1)

In [8]:
df_ohe = pd.get_dummies(df)

In [9]:
df_ohe.shape

(53940, 27)

# Outlier Handling

## Z-score based / Standard Scaler based method

In [10]:
num_columns = df.select_dtypes(exclude='object').columns

In [11]:
num_columns

Index(['carat', 'depth', 'table', 'price', 'x', 'y', 'z'], dtype='object')

In [12]:
def zscore_for_column(df,col):
    #zscore = (val - mean) / std_dev
    zscore = (df[col] - df[col].mean())/df[col].std()
    return zscore

In [14]:
df_std = df_ohe.copy()
for col in num_columns:
    df_std[col] = zscore_for_column(df,col)

In [15]:
df_std.describe()

,carat,depth,table,price,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
count,5.394000e+04,5.394000e+04,5.394000e+04,5.394000e+04,5.394000e+04,5.394000e+04,5.394000e+04,53940.000000,53940.000000,53940.000000,...,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,3.774170e-14,5.528923e-13,-3.621533e-14,-6.602421e-15,1.177763e-13,8.638663e-14,-2.522116e-13,0.029848,0.090953,0.399537,...,0.100519,0.052058,0.013737,0.033185,0.242214,0.170449,0.151483,0.227253,0.067760,0.093919
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.170169,0.287545,0.489808,...,0.300694,0.222146,0.116400,0.179121,0.428427,0.376030,0.358522,0.419061,0.251337,0.291719
min,-1.261446e+00,-1.308748e+01,-6.470013e+00,-9.040868e-01,-5.109073e+00,-5.020884e+00,-5.014510e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-8.395154e-01,-5.231005e-01,-6.521325e-01,-7.476738e-01,-9.103164e-01,-8.882717e-01,-8.909378e-01,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-2.066190e-01,3.531645e-02,-2.046032e-01,-3.839636e-01,-2.777527e-02,-2.147379e-02,-1.237607e-02,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.106635e-01,5.239313e-01,6.904554e-01,3.487834e-01,7.210475e-01,7.052356e-01,7.103118e-01,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.885992e+00,1.204128e+01,1.680151e+01,3.732404e+00,4.465161e+00,4.654922e+01,4.004720e+01,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Detect Outliers

In [16]:
df_std.loc[(df_std['carat'] < -3) | (df_std['carat'] > +3), :].shape

(439, 27)

In [17]:
from IPython.utils.text import columnize
def outlier_imputation(df,col):
    df.loc[df[col] < -3 ,col] = -3
    df.loc[df[col] > 3 ,col] = 3
    return df

In [18]:
for col in num_columns:
    df_std = outlier_imputation(df_std,col)

In [19]:
df_std.describe()

,carat,depth,table,price,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,...,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,-0.004192,-0.002399,-0.005533,-0.007732,0.000079,-0.001072,-0.000280,0.029848,0.090953,0.399537,...,0.100519,0.052058,0.013737,0.033185,0.242214,0.170449,0.151483,0.227253,0.067760,0.093919
std,0.984323,0.951256,0.975847,0.974620,0.997949,0.973317,0.980725,0.170169,0.287545,0.489808,...,0.300694,0.222146,0.116400,0.179121,0.428427,0.376030,0.358522,0.419061,0.251337,0.291719
min,-1.261446,-3.000000,-3.000000,-0.904087,-3.000000,-3.000000,-3.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.839515,-0.523100,-0.652132,-0.747674,-0.910316,-0.888272,-0.890938,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-0.206619,0.035316,-0.204603,-0.383964,-0.027775,-0.021474,-0.012376,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.510664,0.523931,0.690455,0.348783,0.721048,0.705236,0.710312,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## IQR Based outlier handling

In [20]:
df_iqr = df_ohe.copy()

In [21]:
# imputation formula
# IQR = Q3 - Q1
# min_val_value = Q1 - 1.5 * IQR
# max_val_value = Q3 + 1.5 * IQR
# val < min_val_value  --> min_val_value
# val > max_val_value  --> max_val_value

In [23]:
q1, q3 = df['table'].quantile([0.25,0.75])

In [24]:
q1, q3

(56.0, 59.0)

In [26]:
def outlier_imputation_IQR(df,col):
    q1, q3 = df[col].quantile([0.25,0.75])
    iqr = q3 -q1

    df.loc[df[col] < (q1-1.5*iqr),col ] = (q1-1.5*iqr)
    df.loc[df[col] > (q3+1.5*iqr),col ] = (q3+1.5*iqr)
    return df

In [27]:
for col in num_columns:
    df_iqr = outlier_imputation_IQR(df_iqr,col)

In [28]:
df_iqr.describe()

,carat,depth,table,price,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,...,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.792558,61.749156,57.438561,3732.145690,5.731285,5.733311,3.538500,0.029848,0.090953,0.399537,...,0.100519,0.052058,0.013737,0.033185,0.242214,0.170449,0.151483,0.227253,0.067760,0.093919
std,0.457089,1.270254,2.161139,3436.769344,1.119956,1.111950,0.692494,0.170169,0.287545,0.489808,...,0.300694,0.222146,0.116400,0.179121,0.428427,0.376030,0.358522,0.419061,0.251337,0.291719
min,0.200000,58.750000,51.500000,326.000000,1.965000,1.990000,1.215000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,64.750000,63.500000,11885.625000,9.285000,9.270000,5.735000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
